In [1]:
%pylab inline
import pandas as pd
import seaborn
import numpy as np
import matplotlib.pyplot as plt
import scipy
import cv2
import nltk
import string
from collections import defaultdict
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import sys
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectKBest

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier

In [114]:
mile3_df = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_train_df_m3.json')

In [115]:
selected_feats = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "mall_distance", "dt_distance", "manager_id",
                  "pet_policy", "listing_type", "unit_features", "building_features", "apartment",
                  "appliance", "bathroom", "bedroom", "building", "contact", "floor", "high", "kitchen", "room", 
                    "num_photos", "description_len",
                 "managerID_count", "buildingID_count", "weekends", "month_4", "month_5", "created_nights",
                 "created_morning", "created_afternoon", "bed_bath_ratio"]

In [116]:
selected_feats_test = ["bathrooms", "bedrooms", "latitude", "longitude", "price", "mall_distance", "dt_distance", "manager_id",
                  "pet_policy", "listing_type", "unit_features", "building_features", "apartment",
                  "appliance", "bathroom", "bedroom", "building", "contact", "floor", "high", "kitchen", "room", 
                    "num_photos", "description_len",
                 "managerID_count", "buildingID_count", "weekends", "month_4", "month_5", "created_nights",
                 "created_morning", "created_afternoon", "bed_bath_ratio"]

In [117]:
X = mile3_df[selected_feats]
y = mile3_df["interest_level"]

In [118]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)

In [119]:
temp = pd.concat([X_train.manager_id,pd.get_dummies(y_train)], axis = 1).groupby('manager_id').mean()
temp.columns = ['high_frac','low_frac', 'medium_frac']
temp['count'] = X_train.groupby('manager_id').count().iloc[:,1]

In [121]:
temp['manager_skill'] = temp['high_frac']*2 + temp['medium_frac']

In [122]:
# get ixes for unranked managers...
unranked_managers_ixes = temp['count']<20
# ... and ranked ones
ranked_managers_ixes = ~unranked_managers_ixes

In [123]:
mean_values = temp.loc[ranked_managers_ixes, ['high_frac','low_frac', 'medium_frac','manager_skill']].mean()
#print(mean_values)
temp.loc[unranked_managers_ixes,['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values
#print(temp.tail(10))

In [124]:
X_train = X_train.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')

In [125]:
X_val = X_val.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')

In [126]:
new_manager_ixes = X_val['high_frac'].isnull()
X_val.loc[new_manager_ixes,['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values

In [127]:
selected_feats.extend(['high_frac','low_frac', 'medium_frac','manager_skill'])
these_features = [f for f in selected_feats if f not in ['manager_id', 'managerID_count', 'count']]

In [128]:
estimators = [('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
             ('svr', make_pipeline(StandardScaler(),
                               LinearSVC(random_state=42)))]
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

In [129]:
clf.fit(X_train[these_features], y_train).score(X_val[these_features], y_val)

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn

0.7362217066900746

In [130]:
y_val_pred = clf.predict_proba(X_val[these_features])
log_loss(y_val, y_val_pred)

0.5992898301809595

In [137]:
y_train_pred = clf.predict_proba(X_train[these_features])
log_loss(y_train, y_train_pred)

0.3121302753864428

In [131]:
df_test = pd.read_json(r'C:\Users\HP\Documents\cmpt 459 Kaggle\two_sigma\final_test_df_m3.json')
print(df_test.shape)

(74659, 45)


In [91]:
df_test['manager_id'].head()

0    b1b1852c416d78d7765d746cb1b8921f
1    d0b5648017832b2427eeb9956d966a14
2    e6472c7237327dd3903b3d6f6a94515a
3    41735645e0f8f13993c42894023f8e58
4    a742cf7dd3b2627d83417bc3a1b3ec96
Name: manager_id, dtype: object

In [132]:
X = df_test[selected_feats_test]
X = X.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
test_new_manager_ixes = X['high_frac'].isnull()
X.loc[test_new_manager_ixes,['high_frac','low_frac', 'medium_frac','manager_skill']] = mean_values.values

In [133]:
y = clf.predict_proba(X[these_features])

In [135]:
labels2idx = {label: i for i, label in enumerate(clf.classes_)}

In [136]:
sub = pd.DataFrame()
sub["listing_id"] = df_test["listing_id"]
for label in ["high", "medium", "low"]:
    sub[label] = y[:, labels2idx[label]]
sub.to_csv("submission_manageExtra_stack.csv", index=False)